In [1]:
import pandas as pd
import warnings
import os
warnings.filterwarnings('ignore')

In [2]:
parent_dir = os.path.dirname(os.getcwd())
data_path = os.path.join(parent_dir, 'data', 'raw')
out_path = os.path.join(parent_dir, 'data', 'datasets')

In [3]:
matches = pd.read_csv(os.path.join(data_path, 'matchups.csv'))
matches.head()

,P_MATCH_ID,GOLDEARNED,TOTALMINIONSKILLED,WIN,KILLS,ASSISTS,DEATHS,CHAMPION,VISIONSCORE,PUUID,TOTALDAMAGEDEALTTOCHAMPIONS,SUMMONERNAME,GAMEVERSION,Unnamed: 13
0,TR1_1201957752_top,14425,194,False,14,5,7,Kassadin,17,phduyQLB8gBjUerFwiVOtyLLHE9jxw7Jq7dwab_CtRddAv...,33426,Borke,11.14.384.6677,NaN
1,TR1_1201957752_top,12920,177,True,3,13,6,Fiora,17,SgQ5hm74vmXVDB3GIt3ka7gaBXFpNnT3PbHU8TbjEk5PJ8...,12092,Nora Durst,11.14.384.6677,NaN
2,TR1_1201957752_middle,13483,221,False,7,3,6,Yone,20,m7Sxq9HxRKLIgf0n0QOKhfHq8jNMZ-k1VhW_jcb307sUya...,20604,Fightêr Yone,11.14.384.6677,NaN
3,TR1_1201957752_middle,11375,167,True,4,15,7,Corki,27,pa-g2VGIEJn7CJCYYQU1PaHUqjS9pB7M4PsDcjM3ac5ysM...,30150,AM SORİİ HE,11.14.384.6677,NaN
4,TR1_1201957752_bottom,11320,197,False,5,10,12,Kaisa,17,_WeC06P4yIMW7ZMWi6bmwCagXp7c7bxxGv6nNW4--SGqXp...,16689,º J İ N X º,11.14.384.6677,NaN


In [4]:
# splitting the P_MATCH_ID to Region, Match Id and Role
matches[['REGION', 'MATCH_ID', 'ROLE']] = matches['P_MATCH_ID'].str.split('_', expand=True)

# MATCH_ID is not unique across regions. Create a unique identifier.
matches['MATCH_ID'] = matches['MATCH_ID'].str.cat(matches[['REGION']], sep='_')

In [5]:
# Dropping unnecessary columns
matches = matches.drop(columns = ['P_MATCH_ID', 'Unnamed: 13'])

In [6]:
# dropping null records
null_index = matches[matches.isnull().any(axis=1)].MATCH_ID.index.tolist()
matches = matches.drop(index = null_index)
# matches = matches[~matches.MATCH_ID.isin(null_match_id)]

matches[matches.isnull().any(axis=1)].count().sum()

0

In [7]:
# Drop invalid records
numeric_columns = matches._get_numeric_data().columns
invalid_index = matches[matches[numeric_columns].values < 0].MATCH_ID.index.tolist()
matches = matches.drop(index=invalid_index)
# matches = matches[~matches.MATCH_ID.isin(invalid_match_id)]

matches[matches[numeric_columns].values < 0].count().sum()

0

In [8]:
print(matches.columns)
print(matches.shape)

Index(['GOLDEARNED', 'TOTALMINIONSKILLED', 'WIN', 'KILLS', 'ASSISTS', 'DEATHS',
       'CHAMPION', 'VISIONSCORE', 'PUUID', 'TOTALDAMAGEDEALTTOCHAMPIONS',
       'SUMMONERNAME', 'GAMEVERSION', 'REGION', 'MATCH_ID', 'ROLE'],
      dtype='object')
(1312248, 15)


In [9]:
matches.to_csv(os.path.join(out_path, 'dataset.csv'), index=False)